In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'n2o': 3.2e-07}
band = [3]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o n2o band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.8,), (1.66, 1.8)]
  wgt = [(0.9,), (0.5, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.009795                    1  0.011181
0.000750        2  0.010701                    2  0.011888
0.001052        3  0.012586                    3  0.013447
0.001476        4  0.014567                    4  0.015045
0.002070        5  0.016622                    5  0.016705
0.002904        6  0.018753                    6  0.018426
0.004074        7  0.020958                    7  0.020198
0.005714        8  0.023231                    8  0.022031
0.008015        9  0.025583                    9  0.023926
0.011243       10  0.028015                   10  0.025887
0.015771       11  0.030517                   11  0.027911
0.022122       12  0.033081                   12  0.029998
0.031031       13  0.035697                   13  0.032143
0.043528       14  0.038046                   14  0.034093
0.061057       15  0.039845                   15  0.035632
0.085645       16  0.041380                   16  0.036988
0.120136       17  0.042881                   17  0.038364
0.168516       18  0.044359                   18  0.039768
0.236378       19  0.045802                   19  0.041192
0.331549       20  0.047221                   20  0.042624
0.465100       21  0.048434                   21  0.043926
0.652400       22  0.046684                   22  0.042862
0.915100       23  0.041124                   23  0.038706
1.283650       24  0.034734                   24  0.033818
1.800600       25  0.028815                   25  0.029160
2.525700       26  0.023412                   26  0.024695
3.542800       27  0.018559                   27  0.020425
4.969550       28  0.014772                   28  0.016803
6.970850       29  0.012321                   29  0.014069
9.778100       30  0.010514                   30  0.011717
13.715850      31  0.008844                   31  0.009387
19.239350      32  0.007297                   32  0.007194
26.987250      33  0.006484                   33  0.005712
37.855300      34  0.006416                   34  0.004999
53.100050      35  0.006318                   35  0.004503
73.887500      36  0.006011                   36  0.004114
97.662500      37  0.005519                   37  0.003786
121.437500     38  0.004772                   38  0.003344
145.212500     39  0.003915                   39  0.002825
168.987500     40  0.003141                   40  0.002351
192.762500     41  0.002427                   41  0.001908
216.537500     42  0.001748                   42  0.001481
240.312500     43  0.001082                   43  0.001066
264.087500     44  0.000410                   44  0.000660
287.862500     45 -0.000193                   45  0.000338
311.637500     46 -0.000089                   46  0.000592
335.412500     47  0.001004                   47  0.001635
359.187500     48  0.002445                   48  0.002988
382.962500     49  0.003837                   49  0.004338
406.737500     50  0.005167                   50  0.005657
430.512500     51  0.006448                   51  0.006943
454.287500     52  0.007685                   52  0.008197
478.062500     53  0.008883                   53  0.009420
501.837500     54  0.010042                   54  0.010607
525.612500     55  0.011158                   55  0.011744
549.387500     56  0.012227                   56  0.012834
573.162500     57  0.013240                   57  0.013870
596.937500     58  0.014190                   58  0.014845
620.712500     59  0.015070                   59  0.015750
644.487500     60  0.015875                   60  0.016582
668.262500     61  0.016602                   61  0.017336
692.037500     62  0.017260                   62  0.018022
715.812500     63  0.017857                   63  0.018646
739.587500     64  0.018404                   64  0.019216
763.362500     65  0.018911                   65  0.019740
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -23.030142  0.000000 -23.030142 -23.050537  4.464724e-08   
0.000624    2     -23.030143  0.000001 -23.030141 -23.050537  1.170834e-06   
0.000876    3     -23.030143  0.000002 -23.030141 -23.050537  1.636606e-06   
0.001229    4     -23.030143  0.000003 -23.030141 -23.050538  2.322819e-06   
0.001723    5     -23.030144  0.000004 -23.030140 -23.050538  3.332218e-06   
0.002417    6     -23.030144  0.000006 -23.030138 -23.050538  4.816349e-06   
0.003391    7     -23.030145  0.000008 -23.030136 -23.050538  6.997888e-06   
0.004757    8     -23.030145  0.000012 -23.030133 -23.050538  1.020237e-05   
0.006672    9     -23.030145  0.000018 -23.030127 -23.050538  1.490679e-05   
0.009359    10    -23.030146  0.000026 -23.030119 -23.050537  2.181001e-05   
0.013128    11    -23.030146  0.000039 -23.030107 -23.050535  3.193505e-05   
0.018415    12    -23.030144  0.000057 -23.030088 -23.050533  4.677737e-05   
0.025830    13    -23.030142  0.000083 -23.030059 -23.050528  6.852214e-05   
0.036232    14    -23.030136  0.000122 -23.030015 -23.050520  1.003574e-04   
0.050823    15    -23.030127  0.000178 -23.029949 -23.050508  1.467133e-04   
0.071291    16    -23.030111  0.000259 -23.029852 -23.050488  2.136185e-04   
0.100000    17    -23.030086  0.000374 -23.029712 -23.050459  3.097880e-04   
0.140271    18    -23.030046  0.000539 -23.029507 -23.050414  4.479894e-04   
0.196760    19    -23.029984  0.000774 -23.029210 -23.050346  6.465681e-04   
0.275997    20    -23.029888  0.001108 -23.028780 -23.050245  9.318019e-04   
0.387100    21    -23.029740  0.001581 -23.028159 -23.050093  1.341076e-03   
0.543100    22    -23.029512  0.002248 -23.027264 -23.049868  1.927495e-03   
0.761700    23    -23.029201  0.003146 -23.026055 -23.049565  2.734536e-03   
1.068500    24    -23.028839  0.004279 -23.024560 -23.049213  3.789465e-03   
1.498800    25    -23.028458  0.005668 -23.022790 -23.048840  5.140623e-03   
2.102400    26    -23.028086  0.007357 -23.020730 -23.048477  6.863250e-03   
2.949000    27    -23.027777  0.009396 -23.018382 -23.048182  9.044747e-03   
4.136600    28    -23.027628  0.011857 -23.015771 -23.048050  1.178669e-02   
5.802500    29    -23.027753  0.014897 -23.012856 -23.048196  1.524793e-02   
8.139200    30    -23.028266  0.018821 -23.009445 -23.048736  1.968276e-02   
11.417000   31    -23.029386  0.024023 -23.005363 -23.049888  2.538457e-02   
16.014700   32    -23.031530  0.030984 -23.000546 -23.052049  3.265922e-02   
22.464000   33    -23.035346  0.040375 -22.994971 -23.055814  4.192022e-02   
31.510500   34    -23.041459  0.053436 -22.988023 -23.061801  5.402890e-02   
44.200100   35    -23.050461  0.072084 -22.978378 -23.070713  7.045609e-02   
62.000000   36    -23.063536  0.098480 -22.965057 -23.083870  9.310850e-02   
85.775000   37    -23.081657  0.133529 -22.948128 -23.102403  1.232294e-01   
109.550000  38    -23.100537  0.167953 -22.932583 -23.121929  1.534199e-01   
133.325000  39    -23.120497  0.201354 -22.919143 -23.142621  1.835302e-01   
157.100000  40    -23.141751  0.233635 -22.908117 -23.164588  2.134524e-01   
180.875000  41    -23.164264  0.264993 -22.899270 -23.187754  2.432399e-01   
204.650000  42    -23.188023  0.295587 -22.892436 -23.212066  2.729253e-01   
228.425000  43    -23.213053  0.325541 -22.887513 -23.237486  3.025181e-01   
252.200000  44    -23.239414  0.354949 -22.884464 -23.263987  3.320219e-01   
275.975000  45    -23.267207  0.383896 -22.883311 -23.291536  3.614296e-01   
299.750000  46    -23.296446  0.412593 -22.883853 -23.320002  3.908485e-01   
323.525000  47    -23.326221  0.442118 -22.884103 -23.348563  4.210781e-01   
347.300000  48    -23.355098  0.473823 -22.881274 -23.376059  4.531786e-01   
371.075000  49    -23.382444  0.508056 -22.8743

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')